## Dùng word2vec để train, tạo ra embedding  nhờ đó có thể suggest các bài tin tức liên quan nhất tới cụm từ người dùng input.

In [36]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from gensim.models.word2vec import LineSentence
import multiprocessing
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors                                                                            
cores = multiprocessing.cpu_count()
print(cores)

8


In [46]:
import pickle
with open("./data/data_tokened.pickle", "rb") as f:
  data_token = pickle.load(f)

data_list = list(map(lambda x: ' '.join(x), data_token))
print(data_token[:5])
print(data_list[:5])  

[['số_liệu', 'kinh_tế', 'mỹ', 'sẽ', 'tác_động', 'mạnh', 'tới', 'giá', 'vàng', '?', 'giá', 'vàng', 'thế_giới', 'khởi_động', 'tuần', 'giao_dịch', 'mới', 'trong', 'xu_thế', 'giảm', 'nhẹ', 'kéo', 'giá', 'vàng', 'trong', 'nước', 'giảm', 'theo', '_giới', 'phân_tích', 'cho', 'rằng', 'loạt', 'số_liệu', 'quan_trọng', 'của', 'kinh_tế', 'mỹ', 'công_bố', 'trong', 'tuần', 'này', 'có_thể', 'ảnh_hưởng', 'nhiều', 'đến', 'triển_vọng', 'lãi_suất', 'của', 'cục', 'dự_trữ', 'liên_bang', 'mỹ', 'fed', 'từ', 'đó', 'tác_động', 'không', 'nhỏ', 'đến', 'diễn_biến', 'giá', 'vàng', 'lúc', 'gần', 'h', 'trưa', 'nay', 'giá', 'vàng', 'giao', 'ngay', 'tại', 'thị_trường', 'châu_á', 'giảm', 'usd', 'oz', 'so', 'với', 'đóng', 'cửa', 'phiên', 'cuối', 'tuần', 'trước', 'tại', 'mỹ', 'giao_dịch', 'ở', 'mức', 'usd', 'oz', 'mức', 'giá', 'này', 'tương_đương', 'khoảng', 'triệu', 'đồng', 'lượng', 'nếu', 'được', 'quy_đổi', 'theo', 'tỷ_giá', 'usd', 'bán', 'ra', 'tại', 'ngân_hàng', 'vietcombank', 'cùng', 'thời_điểm', 'công_ty', 'bảo', '

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_list)
print(X.shape)

(5019, 46627)


In [8]:
#To proprely work with scikit's vectorizer
merged_questions = [' '.join(question) for question in data_token]
document_names = ['Doc {:d}'.format(i) for i in range(len(merged_questions))]

def get_tfidf(docs, ngram_range=(1,1), index=None):
    vect = TfidfVectorizer(stop_words='english', ngram_range=ngram_range)
    tfidf = vect.fit_transform(docs).todense()
    return pd.DataFrame(tfidf, columns=vect.get_feature_names_out(), index=index).T

tfidf = get_tfidf(merged_questions, ngram_range=(1,1), index=document_names)

In [15]:
from gensim.models import FastText

path = './data/fast_new.model'
ft_model = FastText.load(path)

print(ft_model.wv['kinh_tế'])

[-2.92165041e-01  4.78152722e-01 -1.04250944e+00 -9.89680812e-02
  2.15293452e-01  5.34926243e-02 -3.49078998e-02  1.35056242e-01
  3.74158770e-01 -3.62638861e-01 -2.73760036e-02 -2.20389962e-01
  2.12917641e-01  2.81357408e-01 -1.49783850e-01  3.87781769e-01
 -3.67246240e-01 -4.08129245e-01  5.59060216e-01 -6.49103880e-01
  6.23349920e-02  1.97254598e-01 -3.72478157e-01 -1.97908387e-01
  6.14449680e-02 -2.72229105e-01  2.31160477e-01 -1.17393225e-01
  4.53517705e-01 -1.34577110e-01  2.77455091e-01 -2.31039282e-02
 -2.45647542e-02  2.25192048e-02  2.58990228e-01 -1.73478737e-01
  7.31063948e-04  3.63867767e-02  5.45117743e-02  1.41076520e-01
 -2.67114729e-01 -1.90630928e-01 -3.31837311e-02  4.07149643e-01
 -7.60782287e-02 -3.44041407e-01 -5.05530834e-01 -2.18343750e-01
  8.27507526e-02  2.67352551e-01  6.80616200e-01 -1.00240499e-01
  2.33680215e-02  6.66829152e-03 -1.90532103e-01  5.62540233e-01
 -2.50921607e-01  5.19580722e-01 -1.84901297e-01  3.00185740e-01
  2.94797063e-01  1.98652

In [32]:
n = 100
def get_sent_embs(emb_model):
    sent_embs = []
    for desc in range(len(data_token)):
        sent_emb = np.zeros((1, n))
        if len(data_token[desc]) > 0:
            sent_emb = np.zeros((1, n))
            div = 0
            model = emb_model
            for word in data_token[desc]:
                if word in model.wv.index_to_key and word in tfidf.index:
                    word_emb = model.wv[word]
                    weight = tfidf.loc[word, 'Doc {:d}'.format(desc)]
                    sent_emb = np.add(sent_emb, word_emb * weight)
                    div += weight
                else:
                    div += 1e-13 #to avoid dividing by 0
        if div == 0:
            print(desc)

        sent_emb = np.divide(sent_emb, div)
        sent_embs.append(sent_emb.flatten())
    return sent_embs

In [33]:
ft_sent = get_sent_embs(emb_model = ft_model) 

In [48]:
def get_n_most_similar(interest_index, embeddings, n):
    """
    Takes the embedding vector of interest, the list with all embeddings, and the number of similar questions to 
    retrieve.
    Outputs the disctionary IDs and distances
    """
    nbrs = NearestNeighbors(n_neighbors=n, metric='cosine').fit(embeddings)
    distances, indices = nbrs.kneighbors(embeddings)
    similar_indices = indices[interest_index][1:]
    similar_distances = distances[interest_index][1:]
    return similar_indices, similar_distances

def print_similar(interest_index, embeddings, n):
    """
    Convenience function for visual analysis
    """
    closest_ind, closest_dist = get_n_most_similar(interest_index, embeddings, n)
    print('Question %s \n \n is most similar to these %s questions: \n' % (data_list[interest_index], n))
    for question in closest_ind:
        print('ID ', question, ': ', data_list[question])

In [49]:
print_similar(42, ft_sent, 5)

Question chứng_khoán mỹ giữ đà tăng giá dầu giảm nhẹ thị_trường chứng_khoán mỹ tăng điểm trong phiên giao_dịch ngày thứ_tư khi mùa báo_cáo tài_chính quý tiếp_tục gây hưng_phấn cho nhà_đầu_tư đưa chỉ_số dow_jones đạt chuỗi phiên tăng dài nhất trong gần năm giá dầu_thô giảm nhẹ do áp_lực chốt lời lúc đóng_cửa dow_jones tăng điểm tương_đương đạt điểm chỉ_số s&p tăng đạt điểm chỉ_số nasdaq tăng đạt điểm đây là phiên tăng thứ liên_tiếp của dow_jones đánh_dấu chuỗi phiên tăng dài nhất của chỉ_số này kể từ tháng trong bối_cảnh thiếu_vắng các số_liệu kinh_tế vĩ_mô tâm_điểm chú_ý của nhà_đầu_tư tiếp_tục hướng đến mùa báo_cáo kết_quả kinh_doanh quý ngân_hàng goldman_sachs báo lợi_nhuận không đạt kỳ_vọng nhưng doanh_thu cao hơn dự_báo chốt phiên cổ_phiếu ngân_hàng này tăng gần một_số cổ_phiếu tài chính-ngân hàng khác cũng tăng mạnh như us_bancorp tăng và j.b._hunt tăng đến thời_điểm này mùa báo_cáo tài_chính quý ở phố_wall đã có một sự khởi_đầu mạnh_mẽ trong số các công_ty s&p đã báo_cáo tính đến